- https://www.alignmentforum.org/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream

- later layers had few dead neurons??

In [1]:
# !pip install circuitsvis

In [2]:
import torch
import os

from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner

In [3]:
device = "cuda"

### Model Selection and Evaluation 

In [4]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("tiny-stories-1L-21M") 

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


In [5]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resi

In [6]:
for i in range(5):
    display(
        model.generate(
            "Once upon a time",
            temperature=1,
            verbose=False,
            max_new_tokens=50,
        )
    )

'Once upon a time, there were two friends, Sam and Ben. Ben was not. Ben came to visit Sam when he was a rude boy. He was always very angry and rude.\n\nOne day, Sam and Ben decided to break the wheel while they were'

"Once upon a time, there was an ugly radio that didn't seem so nice anymore! Everyone in the village loved hearing the radio because of the music made it magical. And it was no longer jealous. \n\nOne day, a little girl on how to make"

'Once upon a time, there was a girl called Millie. She was new and very special, because she could mix colors together to make something exciting. She made a cardboard box and planned, and it was the best thing of all!\n\nMillie was so'

'Once upon a time, there was a silly man who lived all alone. One day he decided to go for a walk. He was a bit scared because he heard there were many friendly faces of the tall man. He had long pale hair and a big smile. '

'Once upon a time, there was a wise old man. He had met a three year dependable child. The child said, "I\'m looking for my mommy. Take these, but we don\'t have much money. I know it\'s luxury!" The man'

In [7]:
from transformer_lens.utils import test_prompt

# Test the model with a prompt
test_prompt(
    "Once upon a time, there was a little girl named Lily. She lived in a big, happy little town. On her big adventure,",
    " Lily",
    model,
    prepend_space_to_answer=False,
)

Tokenized prompt: ['<|endoftext|>', 'Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' little', ' girl', ' named', ' Lily', '.', ' She', ' lived', ' in', ' a', ' big', ',', ' happy', ' little', ' town', '.', ' On', ' her', ' big', ' adventure', ',']
Tokenized answer: [' Lily']


Performance on answer token:
Rank: 1        Logit: 18.82 Prob:  8.62% Token: | Lily|

Top 0th token. Logit: 21.00 Prob: 76.18% Token: | she|
Top 1th token. Logit: 18.82 Prob:  8.62% Token: | Lily|
Top 2th token. Logit: 18.16 Prob:  4.45% Token: | there|
Top 3th token. Logit: 17.00 Prob:  1.39% Token: | the|
Top 4th token. Logit: 16.76 Prob:  1.10% Token: | her|
Top 5th token. Logit: 16.61 Prob:  0.94% Token: | all|
Top 6th token. Logit: 16.56 Prob:  0.90% Token: | everyone|
Top 7th token. Logit: 16.04 Prob:  0.53% Token: | things|
Top 8th token. Logit: 16.04 Prob:  0.53% Token: | they|
Top 9th token. Logit: 16.03 Prob:  0.53% Token: | people|


Ranks of the answer tokens: [(' Lily', 1)]

#### Exploring Model Capabilities with Log Probs

In [8]:
import circuitsvis as cv  # optional dep, install with pip install circuitsvis

# Let's make a longer prompt and see the log probabilities of the tokens
example_prompt = """Hi, how are you doing this? I'm really enjoying your posts"""
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)
# hover on the output to see the result.

In [9]:
example_prompt = model.generate(
    "Once upon a time",
    temperature=1,
    verbose=True,
    max_new_tokens=200,
)
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)

  0%|          | 0/200 [00:00<?, ?it/s]

### Training an SAE

In [10]:
total_training_steps = 30_000  # probably we should do more
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="tiny-stories-1L-21M",  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_name="blocks.0.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_layer=0,  # Only one layer in the model.
    d_in=1024,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    streaming=True,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project="sae_lens_tutorial",
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32",
)
# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder = SAETrainingRunner(cfg).run()

Run name: 16384-L1-5-LR-5e-05-Tokens-1.229e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 30000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 30 times.
Number tokens in sparsity calculation window: 4.10e+06


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


wandb: Currently logged in as: lanchunhui (loveresearch). Use `wandb login --relogin` to force relogin











































































































0000| l1_loss: 159.71754 | mse_loss: 199.33046: 100%|██████████| 122880000/122880000 [29:26<00:00, 69580.74it/s]

details/current_l1_coefficient,▁▅██████████████████████████████████████
details/current_learning_rate,████████████████████████████████▇▇▅▅▄▃▂▁
details/n_training_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
losses/l1_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,▅▅█▇▇▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
losses/overall_loss,▁▅█▇▇▆▆▆▆▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
losses/raw_l1_loss,▁▆██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
metrics/explained_variance,▄▄▁▂▃▃▄▅▅▅▅▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█▇▇█
metrics/explained_variance_std,▁▅█▇▇▇▇▆▆▆▆▆▆▆▆▅▆▆▅▅▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▅▅
metrics/l0,█▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/mean_log10_feature_sparsity,█▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
